In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,20 hours 41 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,1 month and 6 days
H2O cluster name:,H2O_from_python_felix_nh59a7
H2O cluster total nodes:,1
H2O cluster free memory:,2.855 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"locked, healthy"


In [15]:
people = h2o.get_frame("people")

In [16]:
people.summary()

,id,bloodType,age,healthyEating,activeLifestyle,income
type,int,enum,real,int,int,int
mins,1.0,,18.0018095958512,0.0,0.0,20500.0
mean,500.5,,41.530897816793335,4.971,5.289999999999998,39533.49999999995
maxs,1000.0,,64.9508899005596,9.0,9.0,65100.0
sigma,288.8194360957494,,13.74867134045404,1.9834548376741756,1.9327792516569589,10729.831174208992
zeros,0,,0,18,9,0
missing,0,0,0,0,0,0
0,1.0,A,63.4590904107317,8.0,4.0,59500.0
1,2.0,A,38.5395801742561,4.0,6.0,32400.0
2,3.0,O,34.5027994732372,8.0,6.0,34700.0


In [17]:
people

id,bloodType,age,healthyEating,activeLifestyle,income
1,A,63.4591,8,4,59500
2,A,38.5396,4,6,32400
3,O,34.5028,8,6,34700
4,O,62.6417,5,6,56600
5,O,60.5844,6,7,54500
6,AB,41.1106,8,4,39600
7,B,25.639,1,8,25900
8,A,32.3888,7,4,32800
9,A,57.1767,3,6,48200
10,A,51.5705,5,5,47300


In [13]:
train, valid, test = people.split_frame(ratios = [0.8, 0.1], 
                                        destination_frames= ['people_train', 'people_valid', 'people_test'],
                                        seed = 123)

In [14]:
print("%d / %d / %d" % (train.nrows, valid.nrows, test.nrows))

788 / 118 / 94


## Make a note of what you will do in regular PANDAS instead of people.split_frame

In [18]:
train = h2o.get_frame("people_train")
valid = h2o.get_frame("people_valid")
test = h2o.get_frame("people_test")

## GBM model

In [19]:
from h2o.estimators import H2OGradientBoostingEstimator

In [38]:
mGBM = H2OGradientBoostingEstimator()

In [39]:
all_cols = train.col_names
print("all columns are: ", all_cols)
y_col = all_cols[len(all_cols)-1]
print("y column is : ", y_col)
x_cols = all_cols[0:len(all_cols)-1]
print("x_columns are : ", x_cols)
print(type(x_cols))
if "id" in x_cols: 
    x_cols.remove("id")
    
print("x_columns are : ", x_cols)
    

all columns are:  ['id', 'bloodType', 'age', 'healthyEating', 'activeLifestyle', 'income']
y column is :  income
x_columns are :  ['id', 'bloodType', 'age', 'healthyEating', 'activeLifestyle']
<class 'list'>
x_columns are :  ['bloodType', 'age', 'healthyEating', 'activeLifestyle']


In [40]:
mGBM.train(x_cols, y_col, model_id = "GBM_for_Artificial_Data_Python", 
           validation_frame = valid,
           training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [41]:
mGBM.model_performance(train = True)


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1200221.245696402
RMSE: 1095.546094738328
MAE: 925.778612785533
RMSLE: 0.031084367908517856
Mean Residual Deviance: 1200221.245696402


In [42]:
mGBM.model_performance(valid = True)


ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 2791298.9401205294
RMSE: 1670.718091157371
MAE: 1422.2945097159359
RMSLE: 0.048042818903929405
Mean Residual Deviance: 2791298.9401205294


In [52]:
perf = mGBM.model_performance(test)
perf


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 2647142.2423868263
RMSE: 1627.0040695667685
MAE: 1356.1590517133443
RMSLE: 0.042358142203433716
Mean Residual Deviance: 2647142.2423868263


In [53]:
mGBM.mae(train = True)

925.778612785533

In [54]:
mGBM.mae(valid = True)

1422.2945097159359

## The below is wrong even when you pass test data to the function

## Need to call the performace object of the test data 

In [55]:
mGBM.mae(test) ### SEE THIS THAT EVEN WITH TEST DATA IT DOESNT WORK

925.778612785533

In [57]:
perf.mae() ### THIS IS THE CORRECT ONE. TO BE GOT FROM THE PERF OBJECT OF TEST DATA

1356.1590517133443

# GBM OVERFITTED

### model_id can be given in H2OGradientBoostingEstimator or in train function

In [63]:
mGBM_overfit = H2OGradientBoostingEstimator(model_id="GBM_Overfit_For_Artificial_Data_Python",
                                            ntrees = 1000,
                                            max_depth = 10)

In [65]:
mGBM_overfit.train(x_cols, y_col, training_frame=train, 
                   validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [67]:
mGBM_overfit.model_performance(train = True)


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 67958.18784545762
RMSE: 260.6879127337085
MAE: 194.21804360326777
RMSLE: 0.007182664314744851
Mean Residual Deviance: 67958.18784545762


In [68]:
mGBM_overfit.model_performance(valid = True)


ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 3547584.2516830196
RMSE: 1883.5031860028853
MAE: 1542.9803287172049
RMSLE: 0.05501491857803932
Mean Residual Deviance: 3547584.2516830196


In [69]:
perf = mGBM_overfit.model_performance(test)
perf


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 3631312.5564373736
RMSE: 1905.600313926657
MAE: 1562.629982376906
RMSLE: 0.048989300746599906
Mean Residual Deviance: 3631312.5564373736


## Cross Validation

In [82]:
mGBM_cv = H2OGradientBoostingEstimator(model_id = "GBM_CV_9_For_Artificial_Data_Python", nfolds = 9)

In [83]:
mGBM_cv.train(x_cols, y_col, training_frame=train) ## no need for validation data

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [84]:
mGBM_cv.model_performance(train = True)


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1200221.245696402
RMSE: 1095.546094738328
MAE: 925.778612785533
RMSLE: 0.031084367908517856
Mean Residual Deviance: 1200221.245696402


In [85]:
mGBM_cv.model_performance(xval= True)


ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 2513083.389805596
RMSE: 1585.2707623007484
MAE: 1350.9868529604296
RMSLE: 0.044138309662255994
Mean Residual Deviance: 2513083.389805596


In [86]:
mGBM_cv.model_performance(test)


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 2647142.2423868263
RMSE: 1627.0040695667685
MAE: 1356.1590517133443
RMSLE: 0.042358142203433716
Mean Residual Deviance: 2647142.2423868263


In [87]:
mGBM_cv_overfit = H2OGradientBoostingEstimator(model_id = "GBM_CV_Overfit_For_Artificial_Data_Python",
                                               nfolds = 9,
                                               ntrees = 1000,
                                               max_depth = 10)

In [88]:
mGBM_cv_overfit.train(x_cols, y_col, train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [89]:
mGBM_cv_overfit.model_performance(train = True)


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 67958.18784545762
RMSE: 260.6879127337085
MAE: 194.21804360326777
RMSLE: 0.007182664314744851
Mean Residual Deviance: 67958.18784545762


In [90]:
mGBM_cv_overfit.model_performance(xval=True)


ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 3327188.714287879
RMSE: 1824.0583089056881
MAE: 1512.9668156416187
RMSLE: 0.04981551505939657
Mean Residual Deviance: 3327188.714287879


In [91]:
mGBM_cv_overfit.model_performance(test)


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 3631312.5564373736
RMSE: 1905.600313926657
MAE: 1562.629982376906
RMSLE: 0.048989300746599906
Mean Residual Deviance: 3631312.5564373736


## Go to flow and check out the following: Models - there will be 10 (K + 1)

### The final model would be trained on the entire training dataset. Validation would have been done on x% (how it decides ???) of the data 